In [ ]:
#@title ZincSight Input {display-mode: "form"}
structure_ids_for_download = '' # @param {type:"string"}
# @markdown - Enter a comma-separated list of structure IDs to fetch.
# @markdown - Permitted IDs include PDB, UniProt, AlphaFold, or ESM identifiers.
# @markdown - For example:
# @markdown   `A0A068N621, A0A0F6AZI6, A0A292DHH8, A0A2U3D0N8, A0A3F2YM30, A0A5H1ZR49, G8ZFK7, O60232, P0A6G5, P0DUH5, P37659, P38164, Q03760, Q08281, Q2K0Z2, Q2UFA9, Q5W0Q7, Q66K64, Q68EN5, Q6CXX6, Q7MVV4, Q86T03, Q8N8R7, Q8NBJ9, Q96JC1, Q9BWG6, Q9D1N4, Q9KP27, Q9M1V3, Q9NUN7, Q9NXF7`
# @markdown - Leave empty if no structures are to be fetched.

upload_structures = False # @param {type: "boolean"}
# @markdown - Check this box in order to upload PDB or MMCIF files individually or in a compressed TAR file

include_rotamers = False # @param {type: "boolean"}
# @markdown - Check this box in order to include rotamer predictions

if upload_structures:
  !mkdir -p /content/query_structures
  %cd /content/query_structures
  from google.colab import files
  uploaded = files.upload()
  %cd /content

In [ ]:
%%capture captured_output
#@title Environment Setup and Imports {display-mode: "form"}
# Install libraries or dependencies
!pip install biopython==1.79 scipy matplotlib psycopg2-binary aria2p python-dotenv scikit-learn joblib
!sudo apt install pymol pigz

# Install Essential System Packages and GPG Tools
!sudo apt install dirmngr ca-certificates software-properties-common gnupg gnupg2 apt-transport-https curl -y

# Install PostgresSQL
!curl -fSsL https://www.postgresql.org/media/keys/ACCC4CF8.asc | gpg --dearmor | sudo tee /usr/share/keyrings/postgresql.gpg > /dev/null
!echo 'deb [arch=amd64,arm64,ppc64el signed-by=/usr/share/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt/ jammy-pgdg main' | sudo tee /etc/apt/sources.list.d/pgdg.list
!sudo apt update
!sudo apt install postgresql-client-15 postgresql-15 -y
!sudo service postgresql start

# Create a new PostgreSQL database if it does not already exist
!sudo -u postgres psql -tc "SELECT 1 FROM pg_database WHERE datname = 'zincsight_pipeline_db';" | grep -q 1 || sudo -u postgres psql -c "CREATE DATABASE zincsight_pipeline_db;"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE zincsight_pipeline_db TO postgres;"
!sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'postgres';"

# Clone the GitHub repository and change into the repo directory
!git clone https://github.com/MECHTI1/ZincSight.git
%cd ZincSight/

# Add essential SQL tables to the established database
!sudo -u postgres psql -d zincsight_pipeline_db -P pager=off -f "src/setup_pg_db_with_tables/PostgreSQL_4_necessary_tables.sql"

In [ ]:
%%capture captured_output
#@title Execute ZincSight {display-mode: "form"}
import os
print("Current Path:", os.getcwd()) # Debugging: Print current working directory

import sys
sys.path.append('/content/ZincSight')
from main_execute import execute_zincsight 

# Define paths explicitly
query_structures_path = '/content/query_structures'  # Path to input structures
output_path = '/content/output'  # Path to save output

# Ensure paths exist
if not os.path.exists(query_structures_path):
    os.makedirs(query_structures_path)
if not os.path.exists(output_path):
    os.makedirs(output_path)  # Create output directory if it doesn't exist

# Call the function with explicit paths
output_compressed_file = execute_zincsight(
    include_rotamers,       # Boolean flag
    structure_ids_for_download,          # String of structure IDs
    query_structures_path,  # Input directory path
    output_path             # Output directory path
)

In [ ]:
#@title Download Result{display-mode: "form"}
from google.colab import files
files.download(output_compressed_file)